## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [63]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
clean_app_df = application_df.drop(columns=["AFFILIATION","EIN","NAME","STATUS","SPECIAL_CONSIDERATIONS"])
clean_app_df.head()

,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,ProductDev,Association,0,5000,1
1,T3,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,C3000,ProductDev,Association,0,5000,0
3,T3,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,C1000,Heathcare,Trust,100000-499999,142590,1


In [64]:
# Determine the number of unique values in each column.
clean_app_df.nunique()

APPLICATION_TYPE      17
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [65]:
# Look at APPLICATION_TYPE value counts for binning
appTypeCounts = pd.value_counts(clean_app_df.APPLICATION_TYPE)
appTypeCounts_df = pd.DataFrame(appTypeCounts)
print(appTypeCounts)
appTypeCounts_df.head()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


,APPLICATION_TYPE
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065


In [66]:
appTypeCounts_df.index[1]

'T4'

In [67]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`


application_types_to_replace = []
for i in range(len(clean_app_df)):
    for j in range(len(appTypeCounts)):
        if clean_app_df["APPLICATION_TYPE"][i]==appTypeCounts_df.index[j]:
            if appTypeCounts_df["APPLICATION_TYPE"][j]>500:
                # print(f"Greater than 500 - {appTypeCounts_df['APPLICATION_TYPE'][j]}")
                application_types_to_replace.append(appTypeCounts_df.index[j])
            else:
                # print(f"Not greater than 500 - {appTypeCounts_df['APPLICATION_TYPE'][j]}")
                application_types_to_replace.append("Other")

# Replace in dataframe
clean_app_df['APPLICATION_TYPE'] = application_types_to_replace

# # Check to make sure binning was successful
clean_app_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [68]:
# Look at CLASSIFICATION value counts for binning
classTypeCounts = pd.value_counts(clean_app_df.CLASSIFICATION)
classTypeCounts_df = pd.DataFrame(classTypeCounts)
print(classTypeCounts)
classTypeCounts_df.head()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


,CLASSIFICATION
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883


In [69]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
for i in range(len(clean_app_df)):
    for j in range(len(classTypeCounts)):
        if clean_app_df["CLASSIFICATION"][i]==classTypeCounts_df.index[j]:
            if classTypeCounts_df["CLASSIFICATION"][j]>1800:
                # print(f"Greater than 500 - {appTypeCounts_df['APPLICATION_TYPE'][j]}")
                classifications_to_replace.append(classTypeCounts_df.index[j])
            else:
                # print(f"Not greater than 500 - {appTypeCounts_df['APPLICATION_TYPE'][j]}")
                classifications_to_replace.append("Other")

# Replace in dataframe
clean_app_df['CLASSIFICATION'] = classifications_to_replace

# # Check to make sure binning was successful
clean_app_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [70]:
# Convert categorical data to numeric with `pd.get_dummies`
clean_app_df_encoded = pd.get_dummies(data=clean_app_df, columns=['APPLICATION_TYPE','CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT'])
clean_app_df_encoded.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [71]:
# Split our preprocessed data into our features and target arrays
y = clean_app_df_encoded.IS_SUCCESSFUL.values
X = clean_app_df_encoded.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [73]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [74]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.


nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=34))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu"))

# # Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=32, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_26 (Dense)            (None, 8)                 280       
                                                                 
 dense_27 (Dense)            (None, 32)                288       
                                                                 
 dense_28 (Dense)            (None, 32)                1056      
                                                                 
 dense_29 (Dense)            (None, 1)                 33        
                                                                 
Total params: 1,657
Trainable params: 1,657
Non-trainable params: 0
_________________________________________________________________


In [75]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [76]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6369 - accuracy: 0.6329
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6154 - accuracy: 0.6504
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6115 - accuracy: 0.6546
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6097 - accuracy: 0.6549
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6090 - accuracy: 0.6555
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.6082 - accuracy: 0.6543
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6079 - accuracy: 0.6572
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6076 - accuracy: 0.6560
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6066 - accuracy: 0.6572
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6068 - accura

In [77]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6152 - accuracy: 0.6526 - 461ms/epoch - 2ms/step
Loss: 0.6152207851409912, Accuracy: 0.6525947451591492


In [78]:
# Export our model to HDF5 file

nn_model.save("AlphabetSoupCharity_optimization4.h5")